<a href="https://colab.research.google.com/github/CodewithTanzeel/RAG_Vector_Embedding_Model/blob/main/RAG_EMBEDDING_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
%pip install -qU langchain-pinecone langchian-google-genai



ERROR: Could not find a version that satisfies the requirement langchian-google-genai (from versions: none)
ERROR: No matching distribution found for langchian-google-genai


In [26]:
from google.colab import userdata

import getpass
import os
!pip uninstall -y pinecone pinecone-plugin-inference
!pip install -U "pinecone<6.0.0"
!pip install langchain-pinecone

from pinecone import Pinecone




pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)


Found existing installation: pinecone 5.4.2
Uninstalling pinecone-5.4.2:
  Successfully uninstalled pinecone-5.4.2
Found existing installation: pinecone-plugin-inference 3.1.0
Uninstalling pinecone-plugin-inference-3.1.0:
  Successfully uninstalled pinecone-plugin-inference-3.1.0
  Using cached pinecone-5.4.2-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone_plugin_inference-3.1.0-py3-none-any.whl.metadata (2.2 kB)
Using cached pinecone-5.4.2-py3-none-any.whl (427 kB)
Using cached pinecone_plugin_inference-3.1.0-py3-none-any.whl (87 kB)


ERROR:pinecone_plugin_interface.logging:Error while installing plugin inference: property 'inference' of 'Pinecone' object has no setter
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pinecone_plugin_interface/actions/installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: property 'inference' of 'Pinecone' object has no setter


In [60]:
pc.delete_index("rag-pinecone-langchain")  # Delete incorrect index


In [65]:
import time
from pinecone import Pinecone, ServerlessSpec

index_name = "rag-pinecone-langchain"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,  # the model we are using have 768 dimensions thats why we have used 768
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)


In [67]:
index_stats = pc.describe_index("rag-pinecone-langchain")
print(index_stats)


{'deletion_protection': 'disabled',
 'dimension': 768,
 'host': 'rag-pinecone-langchain-vz6q933.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'rag-pinecone-langchain',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'},
 'tags': None,
 'vector_type': 'dense'}


In [104]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [102]:
vector = embeddings.embed_query("We are building a RAG Text")


In [103]:
vector[:5]

[0.03732863813638687,
 -0.054927337914705276,
 -0.05723658204078674,
 -0.016060668975114822,
 0.008930283598601818]

In [41]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index,embedding=embeddings)

In [43]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

In [44]:
document_1

Document(metadata={'source': 'tweet'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')

In [45]:
# Data Saving procedure
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [46]:
len(documents)

10

In [52]:

from uuid import uuid4

uuid4()


UUID('de03bdfa-483c-4fe0-af40-cf28b82890b2')

In [74]:


uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents,ids=uuids)
#Delete items from vector store
#vector_store.delete(ids=[uuids[-1]])
# Data Retrieveing


['d2953b3d-9579-4e3d-b6af-d42c9bf7afee',
 'a7be6abb-f786-452c-8056-009a5e5070d5',
 '99a6f86d-b5c6-43ea-bea7-8ad4fe52d7a0',
 'f2a8e3d4-211c-44e9-abea-1d4cd73a459d',
 'ec7caa99-fc4a-4158-9fbd-50599cca7b5c',
 '3a0f2626-a5b3-40a3-82f3-f4f4c3f7bcf3',
 '67bc3f0d-adac-4b9c-b507-1b033dc3fcb6',
 '531146c3-e7f0-4405-adc6-7589e3f8232b',
 '27834792-4202-4554-a1bb-a342ddaee091',
 '504d6ebe-d807-46c7-b3d2-64e99f3a3d68']

In [77]:
# Data Retrival
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
     k=2,
    filter={"source": "tweet"},
 )
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [85]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?",
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [107]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [106]:
def answer_to_user(query:str):

    #Vector Search
    VectorResults = vector_store.similarity_search(query,k=2)
    #Pass to Model Vector Results + User Query
    final_answer = llm.invoke(f"Answer this user query{query},Here are some refrences{VectorResults}")
    return final_answer
    final_answer = ChatGoogleGenerativeAI(results,query)
    return final_answer



In [112]:
answer = answer_to_user("LangChain provides abstractions to make working with LLMs easy")

In [111]:
answer.content

"The provided text mentions LangChain's ease of use for working with LLMs (Large Language Models), but the references are about LangGraph, a different framework.  There's no information in the provided text to answer a question about LangChain itself beyond its stated ease of use.  The references are irrelevant to a query about LangChain."